# Generate augmentations and split files
#### 12 augmentations
#### 1 original
#### Affine transform: Both original image and depth map must be augmented

In [1]:
import os
import os.path as osp
import cv2
import shutil
import random
import pandas as pd
from tqdm.notebook import tqdm
from augmentations import augmentation_choices, transform_image

In [2]:
def make_subdir(subdir):
    if not osp.isdir(subdir):
        try:
            os.mkdir(subdir)
        except FileNotFoundError:
            print("Make sure the parent folder exists")

In [3]:
split_file = 'kitti_eigen_test.txt'
input_prefix_path = "E:\Thesis_Datasets\\kitti_full\\"
input_depth_folder = 'val'
valid_image_count = 0
output_prefix_path = "E:\Thesis_Datasets\\Augmented_Images\\"
output_image_folder = "Data\\"
output_depth_folder = "Depth\\"
image_data = []

In [4]:
valid_image_count = 0
with open(split_file) as f:
    for line in tqdm(f, desc="Augmenting input files...", total=697):
        img_name = line.strip().split(" ")[0]
        img_name = osp.join(input_prefix_path, img_name)
        depth_file = line.strip().split(" ")[1]
        depth_map = osp.join(input_prefix_path, input_depth_folder, depth_file)

        if not osp.isfile(depth_map):
            depth_map = osp.join(input_prefix_path, 'train', depth_file)
        if osp.isfile(img_name) and osp.isfile(depth_map):
            valid_image_count += 1
        
print("Total valid images: ", valid_image_count)

Augmenting input files...:   0%|          | 0/697 [00:00<?, ?it/s]

Total valid images:  652


In [5]:
for aug in augmentation_choices:
    print(aug)

None
gaussian
localvar
poisson
salt
pepper
s&p
speckle
affine
rain
shadow
flare
snow


In [6]:
with open(split_file) as f:
    for line in tqdm(f, desc="Augmenting input files...", total=697):
        img_name = line.strip().split(" ")[0]
        img_name = osp.join(input_prefix_path, img_name)
        depth_file = line.strip().split(" ")[1]
        depth_map = osp.join(input_prefix_path, input_depth_folder, depth_file)
        
        if not osp.isfile(depth_map):
            depth_map = osp.join(input_prefix_path, 'train', depth_file)

        if osp.isfile(img_name) and osp.isfile(depth_map):
            valid_image_count += 1

            # Ensure the folder exists
            make_subdir(osp.join(output_prefix_path, output_depth_folder))

            #   Copy depth map
            shutil.copy(depth_map,
                        osp.join(output_prefix_path, output_depth_folder, depth_map.split('/')[-1]))

            #   For each augmentation type:
            for aug in augmentation_choices:
                if aug == None:
                    aug = "None"
                
                make_subdir(osp.join(output_prefix_path, output_image_folder, aug))

                # if aug = affine, depth map needs to be augmented with the same scale, rotation and shear
                if aug == 'affine':
                    make_subdir(osp.join(output_prefix_path, output_depth_folder, aug))

                    scale=(1.1, 1.1)
                    rotation=random.uniform(-0.05, 0.05)
                    shear=random.uniform(-0.1, 0.1)

                    img = transform_image(aug, cv2.imread(img_name), scale, rotation, shear)
                    augmented_depth = transform_image(aug, cv2.imread(depth_map), scale, rotation, shear)

                    # Save transformed depth map in Depth/affine/xxxxxxx_affine.png
                    cv2.imwrite(osp.join(output_prefix_path, 
                                     output_depth_folder, 
                                     aug, 
                                     img_name[img_name.rfind('/')+1:-4] + "_affine.png"),
                                augmented_depth)
                    
                    # Insert [rgb_file, depth_file, augmentation_type] into list
                    image_data.append([osp.join(output_image_folder, aug, img_name.split('/')[-1]),
                                       osp.join(output_depth_folder, aug, depth_file.split('/')[-1][:-4] + "_affine.png"),
                                       aug])
                # For all other augmentation types:                       
                else:
                    img = transform_image(aug, cv2.imread(img_name))
                    # Insert [rgb_file, depth_file, augmentation_type] into list
                    image_data.append([osp.join(output_image_folder, aug, img_name.split('/')[-1]),
                                       osp.join(output_depth_folder, depth_file.split('/')[-1]),
                                       aug])
                    
                # Save file with new name in /<augmentation_type>/ folder
                cv2.imwrite(osp.join(output_prefix_path, 
                                     output_image_folder, 
                                     aug, 
                                     img_name.split('/')[-1]),
                            img)

print(f"Total augmented files: {valid_image_count}")


Augmenting input files...:   0%|          | 0/697 [00:00<?, ?it/s]

Total augmented files: 1304


In [ ]:
# print(osp.join(output_prefix_path, 
#                                      output_image_folder, 
#                                      aug, 
#                                      img_name.split('/')[-1]))

In [ ]:
# image_data_df['image'][0].replace

In [7]:
# Ensure we are using linux-style '/' instead of windows '\' or '\\'

image_data_df = pd.DataFrame(image_data, columns=['image', 'depth', 'augmentation'])
image_data_df['image'] = image_data_df['image'].apply(lambda x: x.replace('\\', r'/'))
image_data_df['depth'] = image_data_df['depth'].apply(lambda x: x.replace('\\', r'/'))
image_data_df.head(10)

,image,depth,augmentation
0,Data/None/0000000069.png,Depth/0000000069.png,None
1,Data/gaussian/0000000069.png,Depth/0000000069.png,gaussian
2,Data/localvar/0000000069.png,Depth/0000000069.png,localvar
3,Data/poisson/0000000069.png,Depth/0000000069.png,poisson
4,Data/salt/0000000069.png,Depth/0000000069.png,salt
5,Data/pepper/0000000069.png,Depth/0000000069.png,pepper
6,Data/s&p/0000000069.png,Depth/0000000069.png,s&p
7,Data/speckle/0000000069.png,Depth/0000000069.png,speckle
8,Data/affine/0000000069.png,Depth/affine/0000000069_affine.png,affine
9,Data/rain/0000000069.png,Depth/0000000069.png,rain


Create a split_file_<aug>.txt for each augmentation type, and save the full dataframe as a pickle.

In [8]:
for aug in augmentation_choices[1:]:
    subset = image_data_df.loc[image_data_df.augmentation == aug]
    subset[['image', 'depth']].to_csv('E:\Thesis_Datasets\Augmented_Images\split_file_'+aug+'.txt', sep=' ', index=False, header=False)
image_data_df.to_pickle('image_data_df.pkl')

In [12]:
subset = image_data_df.loc[image_data_df.augmentation == 'None']
subset[['image', 'depth']].to_csv('E:\Thesis_Datasets\Augmented_Images\split_file_'+'None.txt', sep=' ', index=False, header=False)

In [11]:
subset.head()

,image,depth,augmentation
0,Data/None/0000000069.png,Depth/0000000069.png,None
13,Data/None/0000000054.png,Depth/0000000054.png,None
26,Data/None/0000000042.png,Depth/0000000042.png,None
39,Data/None/0000000057.png,Depth/0000000057.png,None
52,Data/None/0000000030.png,Depth/0000000030.png,None
